In [57]:
from LoadNBAData import NBADataLoader
import pandas as pd


In [58]:
data = NBADataLoader(load_from_files=True, load_new=True)
data.load_all()
data.enhanced_schedule


No data found for december
No data found for january
No data found for february
No data found for march
No data found for april
No new games to load
Successfully saved nba_data/nba_team_abbreviations.csv
Successfully saved nba_data/schedule_and_results.csv
Successfully saved nba_data/combined_schedule_and_game_stats.csv
Successfully saved nba_data/home_games_stats.csv
Successfully saved nba_data/away_games_stats.csv
Successfully saved nba_data/all_team_stats.csv
Successfully saved nba_data/enhanced_schedule.csv


,Date,Away,AwayPoints,Home,HomePoints,HomeAbbreviation,AwayAbbreviation,GameUrl,A_Team,B_Team,...,Away_advanced_ORB%,Away_advanced_DRB%,Away_advanced_TRB%,Away_advanced_AST%,Away_advanced_STL%,Away_advanced_BLK%,Away_advanced_TOV%,Away_advanced_USG%,Away_advanced_ORtg,Away_advanced_DRtg
0,20241025,Brooklyn Nets,101.0,Orlando Magic,116.0,ORL,BRK,https://www.basketball-reference.com/boxscores...,Brooklyn Nets,Orlando Magic,...,19.000000,85.300000,48.700000,62.900000,7.100000,6.700000,17.100000,100.0,103.000000,118.300000
1,20241025,Philadelphia 76ers,107.0,Toronto Raptors,115.0,TOR,PHI,https://www.basketball-reference.com/boxscores...,Philadelphia 76ers,Toronto Raptors,...,21.700000,50.000000,34.100000,65.500000,10.900000,9.400000,13.900000,100.0,106.100000,114.000000
2,20241025,Charlotte Hornets,120.0,Atlanta Hawks,125.0,ATL,CHO,https://www.basketball-reference.com/boxscores...,Atlanta Hawks,Charlotte Hornets,...,40.700000,81.600000,57.600000,59.500000,6.900000,9.300000,14.100000,100.0,118.500000,123.400000
3,20241025,Detroit Pistons,101.0,Cleveland Cavaliers,113.0,CLE,DET,https://www.basketball-reference.com/boxscores...,Cleveland Cavaliers,Detroit Pistons,...,19.000000,87.200000,51.900000,65.800000,3.000000,10.400000,19.900000,100.0,101.100000,113.100000
4,20241025,Indiana Pacers,98.0,New York Knicks,123.0,NYK,IND,https://www.basketball-reference.com/boxscores...,Indiana Pacers,New York Knicks,...,22.000000,68.600000,43.400000,68.300000,8.400000,10.500000,14.800000,100.0,102.500000,128.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,20241127,Toronto Raptors,119.0,New Orleans Pelicans,93.0,NOP,TOR,https://www.basketball-reference.com/boxscores...,New Orleans Pelicans,Toronto Raptors,...,30.972727,73.554545,51.472727,68.790909,8.054545,8.209091,12.590909,100.0,110.963636,116.945455
258,20241127,Los Angeles Lakers,119.0,San Antonio Spurs,101.0,SAS,LAL,https://www.basketball-reference.com/boxscores...,Los Angeles Lakers,San Antonio Spurs,...,22.266667,73.033333,48.011111,63.922222,7.933333,5.755556,11.511111,100.0,113.411111,118.555556
259,20241127,Brooklyn Nets,127.0,Phoenix Suns,117.0,PHO,BRK,https://www.basketball-reference.com/boxscores...,Brooklyn Nets,Phoenix Suns,...,21.290909,77.445455,48.090909,68.109091,5.763636,6.154545,13.927273,100.0,114.845455,116.254545
260,20241127,Denver Nuggets,122.0,Utah Jazz,103.0,UTA,DEN,https://www.basketball-reference.com/boxscores...,Denver Nuggets,Utah Jazz,...,23.787500,73.012500,49.537500,67.237500,8.837500,7.950000,11.737500,100.0,116.187500,111.637500


In [59]:
data.get_schedule_no_results()
display(data.schedule_no_results)

,Date,Away,AwayPoints,Home,HomePoints
207,"Fri, Nov 29, 2024",New York Knicks,NaN,Charlotte Hornets,NaN
208,"Fri, Nov 29, 2024",Cleveland Cavaliers,NaN,Atlanta Hawks,NaN
209,"Fri, Nov 29, 2024",New Orleans Pelicans,NaN,Memphis Grizzlies,NaN
210,"Fri, Nov 29, 2024",Orlando Magic,NaN,Brooklyn Nets,NaN
211,"Fri, Nov 29, 2024",Los Angeles Clippers,NaN,Minnesota Timberwolves,NaN
...,...,...,...,...,...
99,"Sun, Apr 13, 2025",Utah Jazz,NaN,Minnesota Timberwolves,NaN
100,"Sun, Apr 13, 2025",Oklahoma City Thunder,NaN,New Orleans Pelicans,NaN
101,"Sun, Apr 13, 2025",Los Angeles Lakers,NaN,Portland Trail Blazers,NaN
102,"Sun, Apr 13, 2025",Phoenix Suns,NaN,Sacramento Kings,NaN


In [66]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Prepare the data
df = data.enhanced_schedule

# Select features for prediction
feature_columns = [
    # Team advanced stats
    'Away_advanced_ORtg', 'Away_advanced_DRtg',
    'Home_advanced_ORtg', 'Home_advanced_DRtg',

    # Shooting stats
    'Away_basic_FG%', 'Away_basic_3P%', 'Away_basic_FT%',
    'Home_basic_FG%', 'Home_basic_3P%', 'Home_basic_FT%',

    # Shooting Rates
    'Away_basic_3PA', 'Away_basic_FTA',
    'Home_basic_3PA', 'Home_basic_FTA',

    # Rebounding
    'Away_basic_TRB', 'Away_basic_ORB', 'Away_basic_DRB',
    'Home_basic_TRB', 'Home_basic_ORB', 'Home_basic_DRB',
]
target_columns = ['AwayPoints', 'HomePoints']

X = df[feature_columns].values
y = df[target_columns].values

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Scale both features and targets
feature_scaler = StandardScaler()
target_scaler = StandardScaler()

X_train_scaled = feature_scaler.fit_transform(X_train)
X_test_scaled = feature_scaler.transform(X_test)

y_train_scaled = target_scaler.fit_transform(y_train)
y_test_scaled = target_scaler.transform(y_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train_scaled)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.FloatTensor(y_test_scaled)

# Create Dataset class
class NBADataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create data loaders with smaller batch size
train_dataset = NBADataset(X_train_tensor, y_train_tensor)
test_dataset = NBADataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# Define the neural network with dropout and batch normalization
class NBAPredictor(nn.Module):
    def __init__(self):
        super(NBAPredictor, self).__init__()
        input_size = len(feature_columns)  # Updated for new feature count
        # Reduce layer sizes to prevent overfitting
        self.layer1 = nn.Linear(input_size, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.layer2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.layer3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.layer4 = nn.Linear(32, 2)
        self.dropout = nn.Dropout(0.2)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.dropout(self.relu(self.bn1(self.layer1(x))))
        x = self.dropout(self.relu(self.bn2(self.layer2(x))))
        x = self.dropout(self.relu(self.bn3(self.layer3(x))))
        x = self.layer4(x)
        return x
    # def __init__(self):
    #     super(NBAPredictor, self).__init__()
    #     input_size = len(feature_columns)
        
    #     self.layer1 = nn.Linear(input_size, 256)
    #     self.layer2 = nn.Linear(256, 64)
    #     self.layer3 = nn.Linear(64, 32)
    #     self.layer4 = nn.Linear(32, 2)
    #     self.dropout = nn.Dropout(0.2)
    #     self.relu = nn.ReLU()
        
    # def forward(self, x):
    #     x = self.dropout(self.relu(self.layer1(x)))
    #     x = self.dropout(self.relu(self.layer2(x)))
    #     x = self.dropout(self.relu(self.layer3(x)))
    #     x = self.layer4(x)
    #     return x

# Initialize model, loss function, and optimizer
model = NBAPredictor()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10)

# Training loop with early stopping
num_epochs = 200
best_val_loss = float('inf')
patience = 20
patience_counter = 0

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        total_loss += loss.item()
    
    # Validation phase
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            outputs = model(batch_X)
            val_loss += criterion(outputs, batch_y).item()
    
    avg_val_loss = val_loss / len(test_loader)
    scheduler.step(avg_val_loss)
    
    # Early stopping check
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        # Save best model
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1
        
    if patience_counter >= patience:
        print(f'Early stopping at epoch {epoch+1}')
        break
        
    if (epoch + 1) % 10 == 0:
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_loss:.4f}, Val Loss: {avg_val_loss:.4f}')

# Load best model for evaluation
model.load_state_dict(torch.load('best_model.pth', weights_only=True))
model.eval()

with torch.no_grad():
    test_predictions = model(X_test_tensor)
    # Transform predictions and actual values back to original scale
    test_predictions_original = target_scaler.inverse_transform(test_predictions.numpy())
    y_test_original = target_scaler.inverse_transform(y_test_tensor.numpy())
    
    # Calculate MSE on original scale
    mse = np.mean((test_predictions_original - y_test_original) ** 2)
    rmse = np.sqrt(mse)
    print(f'\nRoot Mean Square Error: {rmse:.2f} points')
    
    # Calculate point differential accuracy
    predicted_diff = test_predictions_original[:, 1] - test_predictions_original[:, 0]
    actual_diff = y_test_original[:, 1] - y_test_original[:, 0]
    correct_direction = np.sum((predicted_diff * actual_diff) > 0)
    accuracy = correct_direction / len(predicted_diff)
    print(f'Point Differential Direction Accuracy: {accuracy:.4f}')


Epoch [10/200], Train Loss: 0.6466, Val Loss: 0.4900
Epoch [20/200], Train Loss: 0.5745, Val Loss: 0.4399
Epoch [30/200], Train Loss: 0.5214, Val Loss: 0.4643
Epoch [40/200], Train Loss: 0.5522, Val Loss: 0.4356
Epoch [50/200], Train Loss: 0.4545, Val Loss: 0.4200
Epoch [60/200], Train Loss: 0.4706, Val Loss: 0.4238
Epoch [70/200], Train Loss: 0.4509, Val Loss: 0.4174
Epoch [80/200], Train Loss: 0.5079, Val Loss: 0.4112
Epoch [90/200], Train Loss: 0.4635, Val Loss: 0.4028
Epoch [100/200], Train Loss: 0.4221, Val Loss: 0.4186
Early stopping at epoch 109

Root Mean Square Error: 7.73 points
Point Differential Direction Accuracy: 0.7037


In [62]:
# Get the most recent game for each team
latest_home_games = data.enhanced_schedule.sort_values('Date').groupby('Home').last().reset_index()
latest_away_games = data.enhanced_schedule.sort_values('Date').groupby('Away').last().reset_index()

# Combine home and away games and get the latest for each team
latest_games = pd.concat([
    latest_home_games[['Home', 'Date']].rename(columns={'Home': 'Team'}).assign(Location='Home'),
    latest_away_games[['Away', 'Date']].rename(columns={'Away': 'Team'}).assign(Location='Away')
])
latest_games = latest_games.sort_values('Date').groupby('Team').last().reset_index()

# print("Most recent game for each team:")
# print(latest_games.sort_values('Date', ascending=False)[['Team', 'Date', 'Location']])


Most recent game for each team:
                      Team      Date Location
0            Atlanta Hawks  20241127     Away
14       Memphis Grizzlies  20241127     Home
28               Utah Jazz  20241127     Home
27         Toronto Raptors  20241127     Away
26       San Antonio Spurs  20241127     Home
25        Sacramento Kings  20241127     Away
24  Portland Trail Blazers  20241127     Away
23            Phoenix Suns  20241127     Home
22      Philadelphia 76ers  20241127     Home
21           Orlando Magic  20241127     Home
20   Oklahoma City Thunder  20241127     Away
19         New York Knicks  20241127     Away
18    New Orleans Pelicans  20241127     Home
17  Minnesota Timberwolves  20241127     Home
15              Miami Heat  20241127     Away
13      Los Angeles Lakers  20241127     Away
12    Los Angeles Clippers  20241127     Away
11          Indiana Pacers  20241127     Home
10         Houston Rockets  20241127     Away
9    Golden State Warriors  20241127     Home
8 

In [63]:
latest_game_stats = pd.DataFrame()
for tup in latest_games.itertuples():
    prefix = tup.Location+"_"
    cols = [col for col in data.enhanced_schedule.columns if str(col).startswith(prefix)]
    latest_game_stats_team = data.enhanced_schedule[(data.enhanced_schedule[tup.Location] == tup.Team) & (data.enhanced_schedule["Date"] == tup.Date)][cols]
    latest_game_stats_team["Team"] = tup.Team
    latest_game_stats_team.rename(columns={col: col.replace(prefix, "") for col in latest_game_stats_team.columns}, inplace=True)
    latest_game_stats = pd.concat([latest_game_stats, latest_game_stats_team])

# display(latest_game_stats)

,basic_MP,basic_FG,basic_FGA,basic_FG%,basic_3P,basic_3PA,basic_3P%,basic_FT,basic_FTA,basic_FT%,...,advanced_DRB%,advanced_TRB%,advanced_AST%,advanced_STL%,advanced_BLK%,advanced_TOV%,advanced_USG%,advanced_ORtg,advanced_DRtg,Team
249,240.000000,43.300000,94.700000,0.459100,13.100000,36.500000,0.358000,16.400000,20.900000,0.780900,...,75.780000,51.730000,71.640000,9.510000,10.030000,14.150000,100.0,112.590000,117.250000,Atlanta Hawks
238,246.250000,41.000000,87.750000,0.468500,19.375000,51.000000,0.385750,15.375000,19.250000,0.795625,...,72.112500,46.950000,66.025000,5.750000,10.437500,11.412500,100.0,119.700000,113.675000,Boston Celtics
259,242.272727,38.000000,82.909091,0.459455,16.272727,41.000000,0.395182,18.454545,21.818182,0.858000,...,77.445455,48.090909,68.109091,5.763636,6.154545,13.927273,100.0,114.845455,116.254545,Brooklyn Nets
248,242.500000,40.000000,90.400000,0.442100,15.900000,43.900000,0.358400,12.400000,16.400000,0.724900,...,76.310000,51.160000,55.710000,7.720000,9.770000,13.930000,100.0,110.210000,112.190000,Charlotte Hornets
251,240.000000,43.454545,90.181818,0.482455,17.454545,43.272727,0.399818,15.545455,19.636364,0.798545,...,78.790909,49.245455,66.918182,7.154545,7.872727,13.163636,100.0,115.654545,117.445455,Chicago Bulls
249,240.000000,46.363636,88.272727,0.526000,15.909091,38.545455,0.412455,15.909091,19.909091,0.774091,...,77.345455,50.363636,62.909091,9.690909,9.727273,12.936364,100.0,122.663636,110.427273,Cleveland Cavaliers
254,240.000000,44.111111,88.888889,0.497111,12.000000,33.666667,0.357667,16.444444,21.333333,0.770333,...,75.266667,51.044444,58.311111,7.844444,10.511111,11.711111,100.0,118.433333,106.255556,Dallas Mavericks
260,246.250000,43.750000,89.250000,0.491625,11.500000,31.000000,0.379250,18.750000,24.375000,0.760875,...,73.012500,49.537500,67.237500,8.837500,7.950000,11.737500,100.0,116.187500,111.637500,Denver Nuggets
255,244.545455,40.000000,88.363636,0.453909,13.545455,37.000000,0.366545,14.818182,19.545455,0.746636,...,79.181818,53.109091,61.045455,6.227273,10.972727,14.627273,100.0,109.636364,110.672727,Detroit Pistons
261,240.000000,42.500000,94.500000,0.450125,16.250000,43.750000,0.370250,13.250000,19.500000,0.689375,...,73.387500,49.512500,70.262500,9.450000,12.775000,12.537500,100.0,112.775000,107.362500,Golden State Warriors


In [64]:
future_games = data.schedule_no_results.merge(
    latest_game_stats.add_prefix('Away_'), 
    left_on="Away", 
    right_on="Away_Team", 
    how="left"
).drop(columns=["Away_Team"])\
.merge(
    latest_game_stats.add_prefix('Home_'), 
    left_on="Home", 
    right_on="Home_Team", 
    how="left"
).drop(columns=["Home_Team"])

In [86]:
# Prepare future games data
future_X = future_games[feature_columns].values

# Scale the features using the same scaler used for training
future_X_scaled = feature_scaler.transform(future_X)
future_X_tensor = torch.FloatTensor(future_X_scaled)

# Make predictions
model.eval()
with torch.no_grad():
    future_predictions = model(future_X_tensor)
    # Transform predictions back to original scale
    future_predictions_original = target_scaler.inverse_transform(future_predictions.numpy())

def predicted_winner(row):
    return row['Home'] if row['PredictedHomePoints'] > row['PredictedAwayPoints'] else row['Away']

def predicted_loser(row):
    return row['Home'] if row['PredictedHomePoints'] < row['PredictedAwayPoints'] else row['Away']

def predicted_spread(row):
    winner_points = max(row['PredictedHomePoints'], row['PredictedAwayPoints'])
    loser_points = min(row['PredictedHomePoints'], row['PredictedAwayPoints'])
    return winner_points - loser_points


# Add predictions to the future_games dataframe
future_games['PredictedAwayPoints'] = future_predictions_original[:, 0]
future_games['PredictedHomePoints'] = future_predictions_original[:, 1]
future_games['PredictedSpread'] = future_games.apply(predicted_spread, axis=1)
future_games['PredictedWinner'] = future_games.apply(predicted_winner, axis=1)

# Display predictions in a readable format
prediction_display = future_games[['Date', 'Away', 'PredictedAwayPoints', 
                                 'Home', 'PredictedHomePoints', 'PredictedWinner', 'PredictedSpread']].copy()

# Round the predictions to 1 decimal place
prediction_display['PredictedAwayPoints'] = prediction_display['PredictedAwayPoints'].round(1)
prediction_display['PredictedHomePoints'] = prediction_display['PredictedHomePoints'].round(1)
prediction_display['PredictedSpread'] = prediction_display['PredictedSpread'].round(1)

# Sort by date and display
prediction_display = prediction_display.sort_values('Date')
prediction_display["Date"] = pd.to_datetime(prediction_display["Date"])

display(prediction_display.sort_values("Date").head(10))

,Date,Away,PredictedAwayPoints,Home,PredictedHomePoints,PredictedWinner,PredictedSpread
0,2024-11-29,New York Knicks,116.500000,Charlotte Hornets,115.400002,New York Knicks,1.1
7,2024-11-29,Toronto Raptors,102.099998,Miami Heat,108.199997,Miami Heat,6.1
8,2024-11-29,Oklahoma City Thunder,119.400002,Los Angeles Lakers,102.500000,Oklahoma City Thunder,16.9
9,2024-11-29,Sacramento Kings,112.599998,Portland Trail Blazers,102.900002,Sacramento Kings,9.7
1,2024-11-29,Cleveland Cavaliers,123.500000,Atlanta Hawks,118.199997,Cleveland Cavaliers,5.4
5,2024-11-29,Boston Celtics,119.400002,Chicago Bulls,114.900002,Boston Celtics,4.6
4,2024-11-29,Los Angeles Clippers,112.599998,Minnesota Timberwolves,116.000000,Minnesota Timberwolves,3.4
3,2024-11-29,Orlando Magic,114.599998,Brooklyn Nets,104.199997,Orlando Magic,10.3
6,2024-11-29,Detroit Pistons,115.500000,Indiana Pacers,117.699997,Indiana Pacers,2.2
2,2024-11-29,New Orleans Pelicans,106.900002,Memphis Grizzlies,119.300003,Memphis Grizzlies,12.5
